# **plsql-rag-engine**

Hi,

PLSQL-RAG-Engine model is a basic type of RAG based retrieval model for the particular Knowledge Base.

## Model Initialization

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B-Instruct-2507")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-4B-Instruct-2507")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

## Required Libraries

In [ ]:
!pip install faiss-cpu
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.0 MB/s eta 0:00:00


In [ ]:
import faiss
import numpy as np
import PyPDF2
import pickle

In [ ]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 19.8 MB/s eta 0:00:00


## Database to Embeddings

In [ ]:
def pdf_to_vect(pdf_path):
  with open(pdf_path, 'rb') as file:
    pdf_reader = PyPDF2.PdfReader(file)
    num_pages = len(pdf_reader.pages)

    pdf_texts = []
    for page_num, page in enumerate(pdf_reader.pages):
      page_text = page.extract_text()
      pdf_texts.append(page_text)

    pdf_text = ' '.join(pdf_texts)

  print('Total Pages: ',num_pages)
  print('Total Text: ',len(pdf_text))

  chunks = []

  for i in range(0, len(pdf_text), 400):
    chunk = pdf_text[i:i+500]
    chunks.append(chunk)

  print('Total Chunks: ',len(chunks))

  with open('chunks.pkl', 'wb') as file:
    pickle.dump(chunks, file)

  embeddings = []
  from sentence_transformers import SentenceTransformer
  model = SentenceTransformer("all-MiniLM-L6-v2") # Changed model to a publicly available one

  for i,chunk in enumerate(chunks):
    print(f'Processing {i+1}/{len(chunks)}')
    embedding = model.encode(chunk)
    embeddings.append(embedding)

  embeddings = np.array(embeddings)
  print('Embeddings Shape: ',embeddings.shape)

  index = faiss.IndexFlatL2(embeddings.shape[1])
  index.add(embeddings)

  faiss.write_index(index, 'index.faiss')
  with open('embeddings.pkl', 'wb') as file:
    pickle.dump(embeddings, file)

  return chunks, embeddings

In [ ]:
pdf_to_vect('/content/PLSQL.pdf')

Total Pages:  50
Total Text:  38577
Total Chunks:  97


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Processing 1/97
Processing 2/97
Processing 3/97
Processing 4/97
Processing 5/97
Processing 6/97
Processing 7/97
Processing 8/97
Processing 9/97
Processing 10/97
Processing 11/97
Processing 12/97
Processing 13/97
Processing 14/97
Processing 15/97
Processing 16/97
Processing 17/97
Processing 18/97
Processing 19/97
Processing 20/97
Processing 21/97
Processing 22/97
Processing 23/97
Processing 24/97
Processing 25/97
Processing 26/97
Processing 27/97
Processing 28/97
Processing 29/97
Processing 30/97
Processing 31/97
Processing 32/97
Processing 33/97
Processing 34/97
Processing 35/97
Processing 36/97
Processing 37/97
Processing 38/97
Processing 39/97
Processing 40/97
Processing 41/97
Processing 42/97
Processing 43/97
Processing 44/97
Processing 45/97
Processing 46/97
Processing 47/97
Processing 48/97
Processing 49/97
Processing 50/97
Processing 51/97
Processing 52/97
Processing 53/97
Processing 54/97
Processing 55/97
Processing 56/97
Processing 57/97
Processing 58/97
Processing 59/97
Proces

(['Complete PL/SQL Learning Guide\nFrom Absolute Basics to Advanced Mastery\nTable of Contents\n▶ Chapter 1: Introduction to PL/SQL\n▶ Chapter 2: PL/SQL Basics - Block Structure\n▶ Chapter 3: Variables and Data Types\n▶ Chapter 4: Control Structures\n▶ Chapter 5: Cursors - Working with Result Sets\n▶ Chapter 6: Exception Handling\n▶ Chapter 7: Procedures\n▶ Chapter 8: Functions\n▶ Chapter 9: Packages\n▶ Chapter 10: Triggers\n▶ Chapter 11: Collections\n▶ Chapter 12: Records and Object Types\n▶ Chapter 13: Dynamic ',
  "10: Triggers\n▶ Chapter 11: Collections\n▶ Chapter 12: Records and Object Types\n▶ Chapter 13: Dynamic SQL\n▶ Chapter 14: Bulk Operations\n▶ Chapter 15: Advanced Cursor Techniques\n▶ Chapter 16: Performance Optimization\n▶ Chapter 17: Security and Best Practices ▶ Chapter 18: Real-World Projects Chapter 1: Introduction to PL/SQL\nWhat is PL/SQL?\nPL/SQL (Procedural Language/Structured Query Language) is Oracle's\nprocedural extension to SQL. It combines the data manipulat

## RAG Model

In [ ]:
def ask_question(question,model,tokenizer):
  index = faiss.read_index('index.faiss')
  with open('chunks.pkl', 'rb') as file:
    chunks = pickle.load(file)

  # chunks = chunks['chunks']

  with open('embeddings.pkl', 'rb') as file:
    embeddings = pickle.load(file)

  from sentence_transformers import SentenceTransformer
  encmodel = SentenceTransformer("all-MiniLM-L6-v2")
  question_embedding = encmodel.encode(question)

  question_embedding = np.array(question_embedding).reshape(1,-1)
  distances, indices = index.search(question_embedding, k=3)

  for i, (distance, index) in enumerate(zip(distances[0], indices[0])):
    print(f"Question: {question}")
    print(f"\nAnswer: {chunks[index]}")
    print(f"\n--------\n")

  prompt = '''You are a bot that make answers about PLSQL.
  you answer only in 50 words.
  this is the recommended input : {relevant_document}
  the user input is: {user_input}
  compile a recommendation to the user based on recommended question and user input'''

  retrieved_docs = "\n".join([chunks[idx] for idx in indices[0]])
  prompt = prompt.format(relevant_document=retrieved_docs, user_input=question)
  inputs = tokenizer(prompt, return_tensors="pt")
  outputs = model.generate(**inputs, max_new_tokens=50)
  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  response_text = generated_text[len(prompt):].strip()
  return response_text

### RAG Model Output

In [ ]:
ask_question('what is PLSQL ?',model,tokenizer)

Question: what is PLSQL ?

Answer: 10: Triggers
▶ Chapter 11: Collections
▶ Chapter 12: Records and Object Types
▶ Chapter 13: Dynamic SQL
▶ Chapter 14: Bulk Operations
▶ Chapter 15: Advanced Cursor Techniques
▶ Chapter 16: Performance Optimization
▶ Chapter 17: Security and Best Practices ▶ Chapter 18: Real-World Projects Chapter 1: Introduction to PL/SQL
What is PL/SQL?
PL/SQL (Procedural Language/Structured Query Language) is Oracle's
procedural extension to SQL. It combines the data manipulation power of SQL
with the processi

--------

Question: what is PLSQL ?

Answer: acle's
procedural extension to SQL. It combines the data manipulation power of SQL
with the processing power of procedural languages.
Why PL/SQL?
Before PL/SQL, developers had to send multiple SQL statements to the
database, which was inefficient. PL/SQL solves this by:
Reducing Network Traffic:  Multiple SQL statements can be bundled
into a single block and sent to the database at once.
Better Performance:  Code r

"PL/SQL is Oracle's procedural extension to SQL, combining both in a single language. It enables developers to write procedural code (loops, conditions) directly in the database, improving performance, security, and portability. Ideal for complex business logic"